In [1]:
pip install tensorflow==2.15.1 

In [2]:
import os
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, TimeDistributed, Bidirectional, Concatenate, GlobalAveragePooling1D, AdditiveAttention
from tensorflow.keras.optimizers import Adam
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [7]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, Dense, TimeDistributed, Bidirectional, LSTM, Concatenate, AdditiveAttention, GlobalAveragePooling1D, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

# Definer modellen som en funksjon for bruk med KerasClassifier
def build_model(conv_filter_units=32, dense_units=100, lstm_units=80, lstm_dropout_rate=0.02, lstm_r_dropout_rate=0.05, learning_rate=0.001, dropout_rate=0.05):
    sequence_input = Input(shape=(input_length,), dtype='int32')
    embedded_sequences = Embedding(input_dim=embeddings_GloVe.shape[0],
                                   output_dim=embeddings_GloVe.shape[1],
                                   weights=[embeddings_GloVe],
                                   trainable=False)(sequence_input)

    conv_blocks = []
    lstm_blocks = []

    for _ in range(parallel_blocks):
        conv = Conv1D(
            filters=conv_filter_units,
            kernel_size=1,
            activation='relu',
            padding='same',
            strides=1)(embedded_sequences)
        conv_dense = TimeDistributed(Dense(dense_units, activation='relu'))(conv)
        conv_blocks.append(conv_dense)

        lstm = Bidirectional(LSTM(
            units=lstm_units,
            return_sequences=True,
            dropout=lstm_dropout_rate,
            recurrent_dropout=lstm_r_dropout_rate
        ))(conv_dense)
        lstm_blocks.append(lstm)

    combined = Concatenate()(conv_blocks + lstm_blocks)
    attention_layer = AdditiveAttention(use_scale=True)
    attention_output = attention_layer([combined, combined], return_attention_scores=False)
    context_vector = GlobalAveragePooling1D()(attention_output)

    dense = Dense(units=dense_units, activation='relu')(context_vector)
    dropout = Dropout(dropout_rate)(dense)
    outdata = Dense(num_classes, activation='sigmoid')(dropout)
    model = Model(inputs=sequence_input, outputs=outdata)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_length = 20
num_classes = 1
parallel_blocks = 2

In [9]:
model= build_model()
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

ValueError: Unrecognized keyword arguments passed to Embedding: {'weights': [array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.29019001,  0.80497003,  0.31187001, ..., -0.33603001,
         0.45998001, -0.11278   ],
       ...,
       [-0.26343   ,  0.34340999,  0.032067  , ..., -0.25034001,
        -0.13135   ,  0.45581001],
       [-0.39555001,  0.067909  , -0.063694  , ..., -0.37257999,
         0.63815999,  0.44885999],
       [ 0.22718   , -0.41553   , -0.15654001, ...,  0.30133   ,
        -0.21064   , -0.58592999]])]}

In [ ]:
best_model.save(r'C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model')

INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


In [ ]:
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 20, 100)              2223500   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                              